In [29]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [30]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [31]:
soup = BeautifulSoup(source, "lxml")

In [32]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [33]:
# In the html parsed prettified source in the above cell I found the class of the table object containing the Toronto postcodes, boroughs and neighborhoods.
# The table contains both the column headers in the "th" tags and the data I'm looking for in the "td" tags

pctable = soup.find('table', class_='wikitable sortable')
print(pctable)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [34]:
# Putting the colunn headers in the colheaders list and making sure the newline chars are removed

colheaders=[]
for colheader in pctable.find_all('th'):
    colheaders.append(colheader.text.replace('\n',''))
colheaders

['Postcode', 'Borough', 'Neighbourhood']

In [35]:
# Finding and printing all the td tags to understand the data

for data in pctable.find_all('td'):
    print(data)

<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td>
<td>M6A</td>
<td

In [36]:
# Using list pc to build a sorted list of all neighborhoods, without the "Not assigned" boroughs 
# Using list nest pc to build the nested lists with the neighborhood data, to be inserted into the pc list
# The hoods list will contain the final neighborhoods
# count keeps track of the current position in the pcnest list
# skippc boolean variable decides whether the pcnest list will have to be inserted in the pc list, this happens if the borough data (count=1) is "Not assigned"

# initialize variables
pc=[]
pcnest=[]
count=0
skippc=False
for data in pctable.find_all('td'):
    
    # don't use data if borough = Not assigned
    if data.text.replace('\n','')=='Not assigned' and count==1:  
        skippc=True
    
    # always append the data to the nested list, decide later if the nested list will be appended to the pc list
    pcnest.append(data.text.replace('\n',''))
    
    # set the neighborhood name to the borough name if the neighborhood name is Not Assigned (if the borough name would have been Not Assigned, the data will not be appended anyway)
    if data.text.replace('\n','')=='Not assigned' and count==2:
        pcnest[-1]=pcnest[-2]
        
    count=count+1
    if count==3:
        # only append the data if skippc is false (in other words when the borough data was "Not assigned")
        if skippc==False:
            pc.append(pcnest)
        # re-initialize variables    
        pcnest=[]
        count=0
        skippc= False
        
# sort the pc list, needed for the next step to combine the neighborhoods for the same postcode/borough        
pc.sort()

# initialize variables
hoods=[]

# loop thru all postcodes in pc list
for i in range(0,len(pc)-1):
    if i>0:
        # from the second lsit item on, check if the postal code/borough is the same as the previous inserted list item in hoods
        # if it is the same, update the previous list item and add the current neighborhood, separated with a comma
        # if it is not the same, just append the neighborhood data
        if pc[i-1][0]==pc[i][0] and pc[i-1][1]==pc[i][1]:
            hoods[-1][-1]=hoods[-1][-1] + ',' + pc[i][2]
        else:
            hoods.append(pc[i])
    else:
        # append the first neighborhood data (i=0)
        hoods.append(pc[i])

print(hoods)



[['M1B', 'Scarborough', 'Malvern,Rouge'], ['M1C', 'Scarborough', 'Highland Creek,Port Union,Rouge Hill'], ['M1E', 'Scarborough', 'Guildwood,Morningside,West Hill'], ['M1G', 'Scarborough', 'Woburn'], ['M1H', 'Scarborough', 'Cedarbrae'], ['M1J', 'Scarborough', 'Scarborough Village'], ['M1K', 'Scarborough', 'East Birchmount Park,Ionview,Kennedy Park'], ['M1L', 'Scarborough', 'Clairlea,Golden Mile,Oakridge'], ['M1M', 'Scarborough', 'Cliffcrest,Cliffside,Scarborough Village West'], ['M1N', 'Scarborough', 'Birch Cliff,Cliffside West'], ['M1P', 'Scarborough', 'Dorset Park,Scarborough Town Centre,Wexford Heights'], ['M1R', 'Scarborough', 'Maryvale,Wexford'], ['M1S', 'Scarborough', 'Agincourt'], ['M1T', 'Scarborough', "Clarks Corners,Sullivan,Tam O'Shanter"], ['M1V', 'Scarborough', "Agincourt North,L'Amoreaux East,Milliken,Steeles East"], ['M1W', 'Scarborough', "L'Amoreaux West,Steeles West"], ['M1X', 'Scarborough', 'Upper Rouge'], ['M2H', 'North York', 'Hillcrest Village'], ['M2J', 'North York

In [37]:
print("Number of neighborhoods: ", len(hoods))

Number of neighborhoods:  102


In [38]:
dfhoods = pd.DataFrame.from_records(hoods, columns=colheaders)
dfhoods

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [39]:
dfhoods.shape

(102, 3)

In [40]:
!wget -O Geo.csv http://cocl.us/Geospatial_data
dfgeo = pd.read_csv("Geo.csv")


--2019-01-18 13:42:08--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-01-18 13:42:08--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-01-18 13:42:09--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.24.197, 107.152.25.197
Connecting to ibm.box.com (ibm.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-01-18 13:42:09--  https://ibm.ent.box.c

In [41]:
dfgeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
dfhoodsgeo = pd.merge(dfhoods, dfgeo, how='inner', left_on='Postcode', right_on='Postal Code')
dfhoodsgeo.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [43]:
dfhoodsgeo.drop('Postal Code', axis=1, inplace = True)
dfhoodsgeo

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [44]:
dfhoodsgeo.shape

(102, 5)

In [45]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [46]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [47]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfhoodsgeo['Latitude'], dfhoodsgeo['Longitude'], dfhoodsgeo['Borough'], dfhoodsgeo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [48]:
# Let's only work with boroughs that contain the word Toronto 

toronto_data = dfhoodsgeo[dfhoodsgeo['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [49]:
# create map of Toronto using latitude and longitude values
map_toronto_boroughs = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_boroughs)  
    
map_toronto_boroughs

In [50]:
CLIENT_ID = 'LYW4Y42IWBRH3155M0RCN1XEVNVCMYZGMFWGQXRZ11V4CO4K' # your Foursquare ID
CLIENT_SECRET = 'P4K2ZWU31HBNZEQ5SPSRYHQZHU4330YSOCCNHJDRHPQKFDQ3' # your Foursquare Secret
VERSION = '20190118' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LYW4Y42IWBRH3155M0RCN1XEVNVCMYZGMFWGQXRZ11V4CO4K
CLIENT_SECRET:P4K2ZWU31HBNZEQ5SPSRYHQZHU4330YSOCCNHJDRHPQKFDQ3


In [51]:
# Let's explore the first neighborhood in our dataframe.
# Get the neighborhood's name.
toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [52]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [53]:
# Now, let's get the top 10 venues that are in The Beaches within a radius of 500 meters
LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LYW4Y42IWBRH3155M0RCN1XEVNVCMYZGMFWGQXRZ11V4CO4K&client_secret=P4K2ZWU31HBNZEQ5SPSRYHQZHU4330YSOCCNHJDRHPQKFDQ3&v=20190118&ll=43.67635739999999,-79.2930312&radius=500&limit=10'

In [54]:
# Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c41d8144434b947c17c40f5'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b8daea1f964a520480833e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d11b941735',
         'name': 'Pub',
         'pluralName': 'Pubs',
         'primary': True,
         'shortName': 'Pub'}],
       'id': '4b8daea1f964a520480833e3',
       'location': {'address': '676 Kingston Rd.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Main St.',
        'distance': 460,
        'formattedAddress': ['676 Kingston Rd. (at Main St.)',
         'Toronto ON M4E 1R4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [55]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [56]:
# Now we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Upper Beaches,Neighborhood,43.680563,-79.292869
3,Beaches Fitness,Gym / Fitness Center,43.680319,-79.290991
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [57]:
# And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [58]:
# Let's create a function to repeat the same process to all the neighborhoods in Toronto with Toronto in their name
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
Riverdale,The Danforth West
India Bazaar,The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Garden District,Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
North Midtown,The Annex,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
Bathurst Quay,CN Tower,Harbourfront West,Island airport,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [60]:
# Let's check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head(100)

(344, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Beaches Fitness,43.680319,-79.290991,Gym / Fitness Center
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop
5,"Riverdale,The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"Riverdale,The Danforth West",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,"Riverdale,The Danforth West",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"Riverdale,The Danforth West",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
9,"Riverdale,The Danforth West",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant


In [61]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",10,10,10,10,10,10
"Bathurst Quay,CN Tower,Harbourfront West,Island airport,King and Spadina,Railway Lands,South Niagara",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Brockton,Exhibition Place,Parkdale Village",10,10,10,10,10,10
Business Reply Mail Processing Centre 969 Eastern,10,10,10,10,10,10
"Cabbagetown,St. James Town",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
"Chinatown,Grange Park,Kensington Market",10,10,10,10,10,10
Christie,10,10,10,10,10,10


In [62]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 119 uniques categories.


In [69]:
# Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
# And let's examine the new dataframe size.
toronto_onehot.shape

(344, 120)

In [72]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bank,Bar,Beer Bar,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pool,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
1,"Bathurst Quay,CN Tower,Harbourfront West,Islan...",0.1,0.1,0.1,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
2,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.00,0.0,0.0,0.00,0.1,0.00,0.0,0.0
3,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,

In [73]:
# Let's confirm the new size
toronto_grouped.shape

(38, 120)

In [75]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0        Steakhouse   0.2
1  Greek Restaurant   0.1
2       Coffee Shop   0.1
3      Noodle House   0.1
4      Concert Hall   0.1


----Bathurst Quay,CN Tower,Harbourfront West,Island airport,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge   0.2
1    Airport Terminal   0.2
2             Airport   0.1
3  Airport Food Court   0.1
4            Boutique   0.1


----Berczy Park----
                venue  freq
0        Cocktail Bar   0.1
1        Liquor Store   0.1
2            Beer Bar   0.1
3  Italian Restaurant   0.1
4          Steakhouse   0.1


----Brockton,Exhibition Place,Parkdale Village----
                  venue  freq
0           Coffee Shop   0.2
1    Italian Restaurant   0.1
2             Pet Store   0.1
3  Caribbean Restaurant   0.1
4                  Café   0.1


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0        Farmers Mark

In [76]:
# Let's put that into a pandas dataframe
# First, let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Seafood Restaurant,Coffee Shop,Plaza,Concert Hall,Hotel,Speakeasy,Noodle House,Greek Restaurant,Fast Food Restaurant
1,"Bathurst Quay,CN Tower,Harbourfront West,Islan...",Airport Terminal,Airport Lounge,Airport,Harbor / Marina,Plane,Boutique,Airport Gate,Airport Food Court,College Gym,Comfort Food Restaurant
2,Berczy Park,Park,Beer Bar,Concert Hall,Liquor Store,Steakhouse,Farmers Market,Italian Restaurant,Museum,Thai Restaurant,Cocktail Bar
3,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Gym,Pet Store,Caribbean Restaurant,Café,Breakfast Spot,Bar,Furniture / Home Store,Italian Restaurant,Cuban Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Garden Center,Fast Food Restaurant,Pizza Place,Comic Shop,Burrito Place,Brewery,Restaurant,Farmers Market,Auto Workshop,Skate Park


In [80]:
# Cluster Neighborhoods
# Run k-means to cluster the neighborhood into 3 clusters.

# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 2, 1, 2, 2, 2, 2, 1], dtype=int32)

In [83]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Neighborhood,Gym / Fitness Center,Pub,Yoga Studio,Dessert Shop,Cocktail Bar,College Gym,Comfort Food Restaurant,Comic Shop
1,M4K,East Toronto,"Riverdale,The Danforth West",43.679557,-79.352188,1,Greek Restaurant,Yoga Studio,Health Food Store,Cosmetics Shop,Brewery,Ice Cream Shop,Italian Restaurant,Fish Market,Fish & Chips Shop,Coffee Shop
2,M4L,East Toronto,"India Bazaar,The Beaches West",43.668999,-79.315572,1,Park,Sushi Restaurant,Liquor Store,Burger Joint,Pub,Brewery,Italian Restaurant,Gym,Ice Cream Shop,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Comfort Food Restaurant,Ice Cream Shop,Bookstore,Fish Market,Sandwich Place,Café,Cheese Shop,Neighborhood,College Gym
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Dim Sum Restaurant,Bus Line,Swim School,Park,Dance Studio,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant


In [85]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
# We can now examine each cluster and determine the discriminating venue categories that distinguish each cluster.
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Downtown Toronto,43.653206,-79.400049,0,Café,Bar,Arts & Crafts Store,Coffee Shop,Mexican Restaurant,Dessert Shop,Organic Grocery,Cocktail Bar,Fish Market,Creperie
31,West Toronto,43.669005,-79.442259,0,Bakery,Bar,Café,Brewery,Bank,Gym / Fitness Center,Supermarket,Middle Eastern Restaurant,Music Venue,Concert Hall


In [89]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,43.676357,-79.293031,1,Coffee Shop,Neighborhood,Gym / Fitness Center,Pub,Yoga Studio,Dessert Shop,Cocktail Bar,College Gym,Comfort Food Restaurant,Comic Shop
1,East Toronto,43.679557,-79.352188,1,Greek Restaurant,Yoga Studio,Health Food Store,Cosmetics Shop,Brewery,Ice Cream Shop,Italian Restaurant,Fish Market,Fish & Chips Shop,Coffee Shop
2,East Toronto,43.668999,-79.315572,1,Park,Sushi Restaurant,Liquor Store,Burger Joint,Pub,Brewery,Italian Restaurant,Gym,Ice Cream Shop,Fish & Chips Shop
4,Central Toronto,43.728020,-79.388790,1,Dim Sum Restaurant,Bus Line,Swim School,Park,Dance Studio,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant
9,Central Toronto,43.686412,-79.400049,1,Pub,Coffee Shop,Fried Chicken Joint,Sushi Restaurant,Convenience Store,Sports Bar,Supermarket,American Restaurant,Comfort Food Restaurant,Dance Studio
11,Downtown Toronto,43.667967,-79.367675,1,Café,Italian Restaurant,Butcher,Indian Restaurant,Restaurant,Diner,Jewelry Store,Japanese Restaurant,General Entertainment,Concert Hall
12,Downtown Toronto,43.665860,-79.383160,1,Gastropub,Bar,Park,Bubble Tea Shop,Ramen Restaurant,Breakfast Spot,Salon / Barbershop,Bookstore,Dance Studio,Tea Room
15,Downtown Toronto,43.651494,-79.375418,1,Gastropub,BBQ Joint,Coffee Shop,Middle Eastern Restaurant,Restaurant,Creperie,Japanese Restaurant,Italian Restaurant,Church,Food Truck
17,Downtown Toronto,43.657952,-79.387383,1,Coffee Shop,Gastropub,Italian Restaurant,Spa,Ramen Restaurant,Sushi Restaurant,Pizza Place,College Gym,Comfort Food Restaurant,Comic Shop
18,Downtown Toronto,43.650571,-79.384568,1,Steakhouse,Seafood Restaurant,Coffee Shop,Plaza,Concert Hall,Hotel,Speakeasy,Noodle House,Greek Restaurant,Fast Food Restaurant


In [90]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(3, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,43.659526,-79.340923,2,Coffee Shop,Comfort Food Restaurant,Ice Cream Shop,Bookstore,Fish Market,Sandwich Place,Café,Cheese Shop,Neighborhood,College Gym
5,Central Toronto,43.712751,-79.390197,2,Park,Gym,Breakfast Spot,Clothing Store,Hotel,Burger Joint,Dance Studio,Sandwich Place,Grocery Store,Food & Drink Shop
6,Central Toronto,43.715383,-79.405678,2,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Chinese Restaurant,Spa,Diner,Salon / Barbershop,Mexican Restaurant,Coffee Shop,Comic Shop
7,Central Toronto,43.704324,-79.388790,2,Dessert Shop,Park,Sushi Restaurant,Café,Pizza Place,Coffee Shop,Italian Restaurant,Seafood Restaurant,Indian Restaurant,Farmers Market
8,Central Toronto,43.689574,-79.383160,2,Park,Tennis Court,Playground,Gym,Clothing Store,Cocktail Bar,Coffee Shop,Church,Dance Studio,College Gym
10,Downtown Toronto,43.679563,-79.377529,2,Park,Trail,Playground,Dance Studio,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant
13,Downtown Toronto,43.654260,-79.360636,2,Breakfast Spot,Pub,Restaurant,Spa,Bakery,Historic Site,Park,Coffee Shop,Gym / Fitness Center,Concert Hall
14,Downtown Toronto,43.657162,-79.378937,2,Burrito Place,Plaza,Clothing Store,Thai Restaurant,Café,Tea Room,Taco Place,Ramen Restaurant,Pizza Place,Comic Shop
16,Downtown Toronto,43.644771,-79.373306,2,Park,Beer Bar,Concert Hall,Liquor Store,Steakhouse,Farmers Market,Italian Restaurant,Museum,Thai Restaurant,Cocktail Bar
27,Downtown Toronto,43.628947,-79.394420,2,Airport Terminal,Airport Lounge,Airport,Harbor / Marina,Plane,Boutique,Airport Gate,Airport Food Court,College Gym,Comfort Food Restaurant
